In [5]:
import zlib
import numpy as np
from datasets import load_dataset
import torch

In [8]:
def entropy_decoder(compressed_img,original_shape):
    decompressed = zlib.decompress(compressed_img)
    ẑ = np.frombuffer(decompressed, dtype=np.int8)
    ẑ = ẑ.reshape(original_shape)
    return ẑ
def batch_entropy_decode(compressed_batch,original_size):
    z = torch.cat([
        torch.tensor(
            entropy_decoder(compressed_batch[i], original_size)
        ) 
        for i in range(batch_size)
    ])
    return z

In [7]:
dataset = load_dataset("danjacobellis/imagenet_RDAE_batched_250k",split='train').with_format("torch")
N_classes=1001;

In [37]:
for (i_batch, batch) in enumerate(dataset):
    latent_size = batch['latent_size']
    batch_size = latent_size[0];
    original_size = latent_size.numpy().copy();
    original_size[0] = 1;
    
    compressed_batch = batch['compressed_batch']
    z = batch_entropy_decode(compressed_batch,original_size);
    z.to("cuda")
    
    label = batch['label'].to(torch.int64)
    y = torch.zeros((batch_size,N_classes,1,1),dtype=torch.int16)
    y.scatter_(1, label.view(-1, 1, 1, 1), 1)
    y.to("cuda")
    
    z.detach()
    y.detach()

CPU times: user 10min 34s, sys: 15.8 s, total: 10min 50s
Wall time: 2min 32s


In [40]:
z.shape

torch.Size([64, 128, 45, 63])

In [41]:
y.shape

torch.Size([64, 1001, 1, 1])

GPT-4 prompt:

The input `z` is a [64, 128, width, height] tensor containing image a batch of 64 (128, width, height) image features. The images are variable sized but each batch contains images of the same size.

The label `y` is a batch of 64 one-hot encoded imagenet labels and has size [64, 1001, 1, 1]

The desired architecture is a fully convolutional network (FCN)
```
64 x  128   x ~128 x ~128 <- input is batches of 64 by 128 by width by height
64 x  256   x ~64  x ~64
64 x  512   x ~32  x ~32
64 x  1024  x ~16  x ~16
64 x  2048  x ~8   x ~8
64 x  4096  x ~4   x ~4
64 x  8192  x ~2   x ~2
64 x 16384  x  1   x  1
64 x  1000  x  1   x  1
```

Write a class Network that inherits from nn.module that implements this network

Output:

To create a Fully Convolutional Network (FCN) that follows the outlined architecture, you can utilize PyTorch's nn.Module as a base class. The FCN will consist of several convolutional layers (nn.Conv2d) with appropriate kernel sizes and strides to reduce the spatial dimensions as needed.

Here's how you can define your networ

```
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        self.conv1 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(512, 1024, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(1024, 2048, kernel_size=3, stride=2, padding=1)
        self.conv5 = nn.Conv2d(2048, 4096, kernel_size=3, stride=2, padding=1)
        self.conv6 = nn.Conv2d(4096, 8192, kernel_size=3, stride=2, padding=1)
        self.conv7 = nn.Conv2d(8192, 16384, kernel_size=2, stride=2, padding=0)  # kernel size 2 and padding 0 to reach 1x1 size
        self.conv8 = nn.Conv2d(16384, 1000, kernel_size=1)  # kernel size 1 to maintain 1x1 size

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = self.conv8(x)  # no activation on the las
```

In the __init__ method, we define the architecture of the network. Each convolutional layer reduces the spatial dimensions of the input roughly by half, due to the stride of 2.

In the forward method, we apply each layer to the input x sequentially, with a ReLU activation function after each layer except for the last one. For the last layer, we don't use an activation function because this is a multi-class classification problem, and we'll likely use a loss function such as CrossEntropyLoss that includes a softmax activation.

Please note that the actual reduction in dimensions may vary depending on the exact sizes of your input images, due to the rounding behavior of the strided convolutions. Also, you'll need to ensure that your input images are large enough to support this many downsampling steps.

Regarding the loading and preprocessing of the data, your current code seems to be doing that correctly. However, you might want to consider moving the data loading and preprocessing code into a PyTorch DataLoader or similar, which can handle batching and shuffling of the data, and potentially speed up training by loading data in parallel.t layer
        return xk: